In [1]:
!pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
df=pd.read_excel('ProductMix_2024-10-01-2024-10-31.xlsx')

In [4]:
# prompt: Remove rows in df where 'modifier group' is 'Wrap Choice',
#     'Soup or Chili',
#     'Salad Choice',
#     'Coffee Choices'

# List of modifier groups to remove
groups_to_remove = ['Wrap Choice', 'Soup or Chili', 'Salad Choice', 'Coffee Choices']

# Remove rows where 'modifier group' is in the list
df = df[~df['Modifier Group'].isin(groups_to_remove)]

In [5]:
# prompt: remove rows from df where item is 'Baked Tortellini', 'Bowl of Beef Chili', 'Coffee', 'Cup of Soup and Salad', 'Cup of Soup of the Day', 'Indiana Cheese & Charcuterie', 'Pimento Cheese ', 'Pita & Hummus' and  'Side Caesar Salad'

items_to_remove = ['Baked Tortellini', 'Bowl of Beef Chili', 'Coffee', 'Cup of Soup and Salad', 'Cup of Soup of the Day', 'Indiana Cheese & Charcuterie', 'Pimento Cheese ', 'Pita & Hummus', 'Side Caesar Salad']
df = df[~df['Item'].isin(items_to_remove)]

In [6]:
# prompt: remove rows from df where 'Modifier' is 'Chocolate No Bake', 'No Salsa', 'No Sour Cream', 'Pub Salad', 'Regular', 'Special K', 'Tomato Wrap', 'Vegan Cheddar Cheese'

modifiers_to_remove = ['Chocolate No Bake', 'No Salsa', 'No Sour Cream', 'Pub Salad', 'Regular', 'Special K', 'Tomato Wrap', 'Vegan Cheddar Cheese']
df = df[~df['Modifier'].isin(modifiers_to_remove)]

In [7]:
df.columns

Index(['Order Date', 'Time Interval', 'Campus', 'Venue', 'Category', 'Item',
       'Modifier Group', 'Modifier', 'Quantity', '% Quantity', 'Sale Price',
       'Subtotal Sale Price', 'Merchant Discount', 'Total Merchant Sale'],
      dtype='object')

In [8]:
df.drop(columns=['Venue','Campus','% Quantity','Sale Price','Subtotal Sale Price','Merchant Discount'], inplace=True)

<ipython-input-8-608676199248>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Venue','Campus','% Quantity','Sale Price','Subtotal Sale Price','Merchant Discount'], inplace=True)


In [9]:
# prompt: convert the column order date to date

df['Order Date'] = pd.to_datetime(df['Order Date'])

<ipython-input-9-dfed6e681ecf>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Order Date'] = pd.to_datetime(df['Order Date'])
<ipython-input-9-dfed6e681ecf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Order Date'] = pd.to_datetime(df['Order Date'])


In [10]:
df['Modifier'].nunique()

58

In [11]:
df['Modifier Group'].nunique()

20

In [12]:
df['Category'].nunique()

9

In [13]:
df['Category'].value_counts()

,count
Category,
Meal Exchange Menu,23006
Entrees,1285
Kids Menu,350
$8.99 Lunch Special,242
Appetizers,108
Beverages,86
Sides,70
Desserts,18
Soups & Salads,1


In [14]:
df['Item'].nunique()

36

In [20]:
# prompt: standardize columns: Sale Price,  Subtotal Sale Price,   Merchant Discount, Total Merchant Sale . Remove the dollar sign, convert them to numerical

def clean_currency(x):
    """If the value is a string, then remove currency symbol and delimiters
    otherwise, the value will be returned as is."""
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '')
    return x

for col in ['Total Merchant Sale']:
    df[col] = df[col].apply(clean_currency).astype(float)

In [21]:
df.isna().sum()

,0
Order Date,0
Time Interval,0
Category,0
Item,0
Modifier Group,0
Modifier,0
Quantity,0
Total Merchant Sale,0


In [22]:
df['Modifier Group'] = df.groupby('Item')['Modifier Group'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown'))
df['Modifier'] = df.groupby('Item')['Modifier'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown'))

In [23]:
# prompt: calculate the total value counts for total merchant sales which contain negative values

# Assuming df is your DataFrame and it has already been processed as in the previous code.
negative_sales_counts = df[df['Total Merchant Sale'] < 0]['Total Merchant Sale'].value_counts()
total_negative_sales = negative_sales_counts.sum()

print(f"Total value counts for negative 'Total Merchant Sale': {total_negative_sales}")

Total value counts for negative 'Total Merchant Sale': 5


In [24]:
# Remove rows with negative Total Merchant Sale
df = df[df['Total Merchant Sale'] >= 0]

In [25]:
from datetime import datetime

# Convert Time Interval to hour manually
df['Hour'] = df['Time Interval'].apply(lambda x: datetime.strptime(x, '%H:%M').hour)

# Verify the changes
print(df[['Time Interval', 'Hour']].tail())

      Time Interval  Hour
25216         21:00    21
25217         21:00    21
25218         21:00    21
25219         21:00    21
25220         21:00    21


<ipython-input-25-88a14652ef26>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hour'] = df['Time Interval'].apply(lambda x: datetime.strptime(x, '%H:%M').hour)


In [26]:
df['day_of_week'] = df['Order Date'].dt.dayofweek  # Monday=0, Sunday=6

<ipython-input-26-79cef4396f79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = df['Order Date'].dt.dayofweek  # Monday=0, Sunday=6


In [27]:
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

<ipython-input-27-20c9fb1c0eec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)


In [28]:
# Lag features for Total Merchant Sale
df['lag_1'] = df['Total Merchant Sale'].shift(1)
df['lag_2'] = df['Total Merchant Sale'].shift(2)

# Rolling average over 3 hours
df['rolling_mean_3'] = df['Total Merchant Sale'].rolling(window=3).mean()

<ipython-input-28-dfc4b53928bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_1'] = df['Total Merchant Sale'].shift(1)
<ipython-input-28-dfc4b53928bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_2'] = df['Total Merchant Sale'].shift(2)
<ipython-input-28-dfc4b53928bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [29]:
# prompt: generate a feature 'rolling_mean_2' which will be rolling average over 2 hours

# Rolling average over 5 hours
df['rolling_mean_5'] = df['Total Merchant Sale'].rolling(window=5).mean()

In [30]:
df.fillna(method='bfill', inplace=True)

<ipython-input-30-168d8f9d469d>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [31]:
df.isna().sum()

,0
Order Date,0
Time Interval,0
Category,0
Item,0
Modifier Group,0
Modifier,0
Quantity,0
Total Merchant Sale,0
Hour,0
day_of_week,0


In [32]:
# prompt: split the data into X and Y (total merchant sales) , not train and test sets

# Define features (X) and target variable (y)
X = df.drop('Total Merchant Sale', axis=1)
y = df['Total Merchant Sale']

In [33]:
target_encoder = ce.TargetEncoder(cols=['Item', 'Modifier'])
X_encoded = target_encoder.fit_transform(X, y)

# Fit one-hot encoder on training set only
onehot_encoder = ce.OneHotEncoder(cols=['Category', 'Modifier Group'])
X_encoded = onehot_encoder.fit_transform(X_encoded)

In [34]:
X_encoded = X_encoded.drop(columns=['Order Date'])

In [35]:
X_encoded.head()

,Time Interval,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,...,Modifier Group_21,Modifier,Quantity,Hour,day_of_week,is_weekend,lag_1,lag_2,rolling_mean_3,rolling_mean_5
0,11:00,1,0,0,0,0,0,0,0,0,...,0,0.859389,2,11,1,0,2.0,2.0,0.666667,0.4
1,11:00,1,0,0,0,0,0,0,0,0,...,0,0.000000,2,11,1,0,2.0,2.0,0.666667,0.4
2,11:00,1,0,0,0,0,0,0,0,0,...,0,0.000000,2,11,1,0,0.0,2.0,0.666667,0.4
3,11:00,1,0,0,0,0,0,0,0,0,...,0,0.000000,2,11,1,0,0.0,0.0,0.000000,0.4
4,11:00,1,0,0,0,0,0,0,0,0,...,0,0.000000,2,11,1,0,0.0,0.0,0.000000,0.4


In [36]:
# prompt: drop column 'Time Interval'

X_encoded = X_encoded.drop(columns=['Time Interval'])
X_encoded.head()

,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,Item,...,Modifier Group_21,Modifier,Quantity,Hour,day_of_week,is_weekend,lag_1,lag_2,rolling_mean_3,rolling_mean_5
0,1,0,0,0,0,0,0,0,0,0.097583,...,0,0.859389,2,11,1,0,2.0,2.0,0.666667,0.4
1,1,0,0,0,0,0,0,0,0,0.097583,...,0,0.000000,2,11,1,0,2.0,2.0,0.666667,0.4
2,1,0,0,0,0,0,0,0,0,0.097583,...,0,0.000000,2,11,1,0,0.0,2.0,0.666667,0.4
3,1,0,0,0,0,0,0,0,0,0.097583,...,0,0.000000,2,11,1,0,0.0,0.0,0.000000,0.4
4,1,0,0,0,0,0,0,0,0,0.097583,...,0,0.000000,2,11,1,0,0.0,0.0,0.000000,0.4


In [37]:
X_encoded.columns

Index(['Category_1', 'Category_2', 'Category_3', 'Category_4', 'Category_5',
       'Category_6', 'Category_7', 'Category_8', 'Category_9', 'Item',
       'Modifier Group_1', 'Modifier Group_2', 'Modifier Group_3',
       'Modifier Group_4', 'Modifier Group_5', 'Modifier Group_6',
       'Modifier Group_7', 'Modifier Group_8', 'Modifier Group_9',
       'Modifier Group_10', 'Modifier Group_11', 'Modifier Group_12',
       'Modifier Group_13', 'Modifier Group_14', 'Modifier Group_15',
       'Modifier Group_16', 'Modifier Group_17', 'Modifier Group_18',
       'Modifier Group_19', 'Modifier Group_20', 'Modifier Group_21',
       'Modifier', 'Quantity', 'Hour', 'day_of_week', 'is_weekend', 'lag_1',
       'lag_2', 'rolling_mean_3', 'rolling_mean_5'],
      dtype='object')

In [38]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both training and testing data
X_scaled = scaler.fit_transform(X_encoded)

In [39]:
X_scaled.shape

(25161, 40)

In [40]:
# prompt: download X_scaled as csv

import pandas as pd

# Assuming X_scaled is already defined from the previous code
X_scaled_df = pd.DataFrame(X_scaled, columns=X_encoded.columns)

# Save the DataFrame to a CSV file
from google.colab import files
X_scaled_df.to_csv('X_scaled.csv', encoding = 'utf-8-sig')
files.download('X_scaled.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
# # prompt: Download y(total merchant sales)

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import category_encoders as ce
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, precision_score, recall_score
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from datetime import datetime
# from google.colab import files
# !pip install category_encoders

# df=pd.read_excel('ProductMix_2024-10-01-2024-10-31.xlsx')
# df.drop(columns=['Venue','Campus','% Quantity','Sale Price'], inplace=True)

# df['Order Date'] = pd.to_datetime(df['Order Date'])

# def clean_currency(x):
#     """If the value is a string, then remove currency symbol and delimiters
#     otherwise, the value will be returned as is."""
#     if isinstance(x, str):
#         return x.replace('$', '').replace(',', '')
#     return x

# for col in ['Subtotal Sale Price', 'Merchant Discount', 'Total Merchant Sale']:
#     df[col] = df[col].apply(clean_currency).astype(float)
# df.isna().sum()
# df['Modifier Group'] = df.groupby('Item')['Modifier Group'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown'))
# df['Modifier'] = df.groupby('Item')['Modifier'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown'))

# # Assuming df is your DataFrame and it has already been processed as in the previous code.
# negative_sales_counts = df[df['Total Merchant Sale'] < 0]['Total Merchant Sale'].value_counts()
# total_negative_sales = negative_sales_counts.sum()

# print(f"Total value counts for negative 'Total Merchant Sale': {total_negative_sales}")
# # Remove rows with negative Total Merchant Sale
# df = df[df['Total Merchant Sale'] >= 0]

# # Convert Time Interval to hour manually
# df['Hour'] = df['Time Interval'].apply(lambda x: datetime.strptime(x, '%H:%M').hour)

# # Verify the changes
# print(df[['Time Interval', 'Hour']].tail())
# df['day_of_week'] = df['Order Date'].dt.dayofweek  # Monday=0, Sunday=6
# df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
# # Lag features for Total Merchant Sale
# df['lag_1'] = df['Total Merchant Sale'].shift(1)
# df['lag_2'] = df['Total Merchant Sale'].shift(2)

# # Rolling average over 3 hours
# df['rolling_mean_3'] = df['Total Merchant Sale'].rolling(window=3).mean()

# # Rolling average over 5 hours
# df['rolling_mean_5'] = df['Total Merchant Sale'].rolling(window=5).mean()
# df.fillna(method='bfill', inplace=True)
# df.isna().sum()

# # Define features (X) and target variable (y)
# X = df.drop('Total Merchant Sale', axis=1)
# y = df['Total Merchant Sale']

# Save the target variable (y) to a CSV file
y.to_csv('y_total_merchant_sales.csv', encoding='utf-8-sig', index=False)
files.download('y_total_merchant_sales.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>